# Learning to Drive in Adverse Weather Conditions

## Introduction & Setup

This project is designed to explore using Reinforcement learning to teach an autonomous agent to drive in adverse weather conditions.

### Setup

- The project will be performed using an autonomous drving simulator called CARLA.
- Python 3.8
- Anconda

[Project Github](https://github.com/rbuckley25/Tempestas)

### Image Segmentor

In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np

In [52]:
def ImageLoader(weather,town):
    dirt = './Datasets/'+weather+'/'+town
    sem_dir = os.listdir(dirt+'/Semantic')
    rgb_dir = os.listdir(dirt+'/RGB')
    sem = []
    rgb = []
    #extract sets of image matrixs for each episode
    for i in range(len(sem_dir)):
        sem.append(np.load(dirt+'/Semantic/'+sem_dir[i]))
        rgb.append(np.load(dirt+'/RGB/'+sem_dir[i]))
    
    return sem, rgb

    
    

In [53]:
semantic, rgb = ImageLoader('ClearNoon','Town04')

In [56]:
semantic[9]

In [ ]:
Image

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [9]:
class PerceptionNet(nn.Module):

    def __init__(self):
        super(PerceptionNet,self).__init__()
        self.conv1 = nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1)
        
        self.conv2 = nn.Conv2d(64, 32, kernel_size=4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(128, 16, kernel_size=4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.conv4 = nn.Conv2d(256, 8, kernel_size=4, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        
        self.conv5 = nn.Conv2d(512, 4, kernel_size=4, stride=2, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        
        self.conv6 = nn.Conv2d(64, 1, kernel_size=4, stride=1)
        
        self.conv7 = torch.nn.ConvTranspose2d(512,4, kernel_size =4, stride=1)
        self.bn6 = nn.BatchNorm2d(512)
        
        self.conv8 = torch.nn.ConvTranspose2d(256,8, kernel_size =4, stride=1, padding=1)
        self.bn7 = nn.BatchNorm2d(256)
        
        self.conv9 = torch.nn.ConvTranspose2d(128,16, kernel_size =4, stride=1, padding=1)
        self.bn8 = nn.BatchNorm2d(128)
        
        self.conv10 = torch.nn.ConvTranspose2d(64,32, kernel_size =4, stride=1, padding=1)
        self.bn9 = nn.BatchNorm2d(64)
        
        self.conv11 = torch.nn.ConvTranspose2d(32,64, kernel_size =4, stride=1, padding=1)
        self.bn10 = nn.BatchNorm2d(64)
        
        self.conv12 = torch.nn.ConvTranspose2d(13,128, kernel_size =4, stride=1, padding=1)
        
            

    def forward(self, x):
        x = x.to(device)
        x = F.leaky_relu(self.conv1(x),negative_slope=0.02)
        x = F.leaku_relu(self.bn2(self.conv2(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn3(self.conv3(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn4(self.conv4(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn5(self.conv5(x)),negative_slope=0.02)
        x = self.conv6(x)
        
        y = F.leaky_relu(self.bn6(self.conv7(x)),negative_slope=0.02)
        y = F.leaky_relu(self.bn7(self.conv8(y)),negative_slope=0.02)
        y = F.leaky_relu(self.bn8(self.conv9(y)),negative_slope=0.02)
        y = F.leaky_relu(self.bn9(self.conv10(y)),negative_slope=0.02)
        y = F.leaky_relu(self.bn10(self.conv11(y)),negative_slope=0.02)
        y = F.leaky_relu(self.conv5(y),negative_slope=0.02)
        y = F.sigmoid(self.bn5(self.conv5(y)))
      
        
        return x, y

In [10]:
model = PerceptionNet()

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)